In [4]:
import os
import sys
import time
import tensorflow as tf

os.chdir('/scratch/edk202/word2gm-fast/notebooks')
os.chdir("..")

from src.word2gm_fast.dataprep.corpus_to_dataset import make_dataset
from src.word2gm_fast.dataprep.dataset_to_vocab import make_vocab

In [5]:
!python -m unittest -b tests.test_corpus_to_dataset

2025-06-20 16:12:28.899800: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-20 16:12:28.914952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750450348.932149 2706504 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750450348.937096 2706504 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750450348.950603 2706504 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [6]:
!python -m unittest -b tests.test_dataset_to_vocab

2025-06-20 16:12:33.884697: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-20 16:12:33.900191: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750450353.917421 2706609 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750450353.922339 2706609 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750450353.935912 2706609 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [13]:
# Corpus file information
corpus_file = "1900.txt"
corpus_dir = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data"
corpus_path = os.path.join(corpus_dir, corpus_file)
file_size = os.path.getsize(corpus_path) / 1024 / 1024
print("CORPUS FILE: ", corpus_path, "\n")

# Load and filter the corpus
start = time.time()
dataset, _ = make_dataset(corpus_path)
duration_load = time.time() - start
rate_load = file_size / duration_load
dataset = dataset.cache()

# Build the vocabulary from the dataset
start = time.time()
vocab_table = make_vocab(dataset)
duration_vocab = time.time() - start
rate_vocab = file_size / duration_vocab

# Benchmarking output
print("[--   Benchmarks   --]\n")
print(f"{'Step':<35}{'Duration':>10}{'Quantity':>21}{'Rate':>21}")
print("-" * 87)
print(f"{'Corpus loading and filtering':<35}{duration_load:8,.2f}{'s':>2}{file_size:18,.2f}{'MB':>3}{rate_load:16,.2f}{'MB/s':>5}")
print(f"{'Vocabulary creation':<35}{duration_vocab:8,.2f}{'s':>2}{file_size:18,.2f}{'MB':>3}{rate_vocab:16,.2f}{'MB/s':>5}")

# Show sample content
print("\n\n[--  Sample Lines  --]\n")
for line in dataset.shuffle(1000).take(5):
    print(line.numpy().decode("utf-8"))

# Test the vocab table with example words
print("\n\n[--   Test Words   --]\n")
test_words = ["UNK", "man", "king", "nonexistentword"]
ids = vocab_table.lookup(tf.constant(test_words)).numpy()
print(f"{'Word':<18} {'ID':>6}")
print("-" * 25)
for word, idx in zip(test_words, ids):
    print(f"{word:<18} {idx:>6}")

CORPUS FILE:  /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data/1900.txt 

[--   Benchmarks   --]

Step                                 Duration             Quantity                 Rate
---------------------------------------------------------------------------------------
Corpus loading and filtering           0.23 s            490.54 MB        2,157.15 MB/s
Vocabulary creation                  589.39 s            490.54 MB            0.83 MB/s


[--  Sample Lines  --]

UNK one act UNK rudeness
UNK one assert UNK claim
UNK one affords UNK secure
UNK one another UNK friendly
UNK one appoint UNK god


[--   Test Words   --]

Word                   ID
-------------------------
UNK                     0
man                 27449
king                24785
nonexistentword         0


2025-06-20 16:27:33.821857: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
